# Programming Assignment 2

In this assignment, you will:

1. Practice writing code to train and evaluate models using both the two-way holdout method, and an evaluation approach appropriate for models with hyperparameters that uses k-fold cross validation plus a test set.

2. Practice writing code to optimize a machine learning model. In particular, we will use gradient descent to optimize a logistic regression model.

3. **574 Only**: Perform optimization with a different algorithm (Newton-Raphson)

# Resources you can use to complete this assignment (a COMPLETE list)

**NOTE: You ARE allowed to use Google to find things that fit this list (i.e. it is often easy to google something like "plotly draw line graph" to find the right part of the plotly documentation).**

- Anything linked to in this article
- Anything linked to from the course web page
- Any materials from another online course taught at a university (**if you use this, you MUST provide a link to the exact document used**)
- Anything posted by Kenny, Navid, or Yincheng on Piazza



# Grading

For grading of code in Part 2, we will execute the submitted notebook as follows:

```shell
jupyter nbconvert --to python Assignment2-Student.ipynb
python Assignment2-Student.py
```

The PDF that comes along with this document has other details on the points awarded for each part.

As such, you will submit, one member of your group will subit as a zip file on UBLearns, a ```.zip``` file that contains 4 things:
- Your completed jupyter notebook.
- Your written report, answering all questions asked here (and copied in the assignment PDF)
- `part_1.1_results.csv`
- `part_1.4_results.csv`


# Part 1 - Predicting Review Scores on Pitchfork

For Part 1, we will be using data from [this paper](https://ojs.aaai.org/index.php/ICWSM/article/view/7355). The data is a collection of reviews from [Pitchfork](https://pitchfork.com/), a site that provides expert reviews of music album. The authors of this paper have also combined the data with a set of features from [Spotify’s API](https://developer.spotify.com/documentation/web-api/) that provide insight into the music itself, e.g. the "acousticness" of the song.  We will tackle a regression problem here, trying to predict the score of a review from several of the other columns in the dataset.

## Part 1.1 - Feature Engineering with Feature Subsets

In the first subsection of Part 1, we’re going to rely on our old friend linear regression. We’re going to look at how running linear regression with various subsets of our features impacts our ability to predict score.

In Part 1.1, your task is to write code below that trains a separate linear regression model for a number of different feature subsets.  Specifically:

- The list `feature_sets` below is a list of lists; each sublist is a different subset of features to build a model with. 
- All models should be trained on the dataset `part1_train.csv`. 
- For each of these trained models, you should evaluate the model’s predictions on the training dataset, as well as the provided test set, called `part1_test.csv`. The evaluation metric we will use is **root mean squared error**.  

Write out the result to a file called `part_1.1_results.csv` and submit this along with your assignment. The file should have the following columns:
- `feature_set` - a column describing the features of the model used. For feature sets with multiple features, combine them using an underscore (you can do this with the code `"_".join(feature_set)`)
- `training_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the training data
- `test_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the test data

In addition, please answer the following questions:
- **1.1.1** Which model had the best RMSE on the *training data*? 
- **1.1.2** Which model had the best RMSE on the *test data*? 
- **1.1.3** Which feature do you believe was the most important one? Why? *(Note: There is more than one perfectly acceptable way to answer this question)*
- **1.1.4** What can we say about the utility of the Spotify features based on these results?

In [22]:
feature_sets = [['artist'],
 ['reviewauthor'],
 ['releaseyear'],
 ['recordlabel'],
 ['genre'],
 ['danceability'],
 ['energy'],
 ['key'],
 ['loudness'],
 ['speechiness'],
 ['acousticness'],
 ['instrumentalness'],
 ['liveness'],
 ['valence'],
 ['tempo'],
 ['danceability','energy','key','loudness','speechiness','acousticness',
  'instrumentalness','liveness','valence','tempo'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre', 'danceability', 
  'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
  'liveness', 'valence', 'tempo']]
import os
print(os.getcwd())

c:\Users\17164\Desktop\Python\ML_Assignment\assignment2


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import validation_curve, learning_curve

# Convenience things for you, note that releaseyear is continuous but is not a Spotify API variable
#os.chdir('D://Games//PythonProgramming//assignment2')
CONTINUOUS_FEATURES = ['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']
CATEGORICAL_FEATURES = ['artist', 'reviewauthor', 'recordlabel', 'genre']

# Read in the data
training_data = pd.read_csv("part1_train.csv")
scaler = StandardScaler()
encoder = OneHotEncoder(sparse=False,handle_unknown='ignore')
y_training = training_data['score']
test_data = pd.read_csv("part1_test.csv")

y_test = test_data['score']

regression = LinearRegression()
rmse_train = []
rmse_test = []
features_for_csv = []
for feature_set in feature_sets:
    #Write your code for Part 1.1 here!
    features_for_csv.append("_".join(feature_set))
    continuous = [feature for feature in feature_set if feature in CONTINUOUS_FEATURES]
    categorical = [feature for feature in feature_set if feature in CATEGORICAL_FEATURES]
    ct = make_column_transformer((scaler,continuous),(encoder,categorical),remainder='passthrough')
    training_data_feature = training_data[continuous+categorical]
    training_data_feature_scaled = ct.fit_transform(training_data_feature)
    testing_data_feature = test_data[continuous+categorical]
    testing_data_feature_scaled = ct.transform(testing_data_feature)
#     print(training_data_feature_scaled.shape,testing_data_feature_scaled.shape)
    regression.fit(training_data_feature_scaled,y_training)
    prediction_on_train = regression.predict(training_data_feature_scaled)
    rmse_train.append(mean_squared_error(prediction_on_train,y_training))
    prediction_on_test = regression.predict(testing_data_feature_scaled)
    rmse_test.append(mean_squared_error(prediction_on_test,y_test))
    pass
rmse_train = np.array(rmse_train)
rmse_test = np.array(rmse_test)
features_for_csv = np.array(features_for_csv)
df = pd.DataFrame({'feature_set':features_for_csv,'rmse_train':rmse_train,'rmse_test':rmse_test})
df.to_csv('part_1.1_results')

## Part 1.2 - Feature Engineering with the LASSO

In Part 1.2, your task is to write code below that trains an L1-regularized linear regression model, with an expanded feature set.  Specifically:

1. Begin with the final feature set listed in `feature_sets` (i.e. your feature set, to begin this section, is `feature_sets[-1]`.
2. One-hot encode your categorical variables, setting `drop=if_binary` and `sparse=False` in the function arguments. 
3. Scale all of your continuous features using the `StandardScaler`.
4. Train an L1-regularized linear regression model using these features on the dataset `part1_train.csv`. You should use the [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) class in `sklearn`, it will do the cross-validation necessary to select the appropriate value for the regularizer for you!  Use 10-fold cross-validation to perform model selection (set the `LassoCV` parmaeter `cv` to 10), and set the `random_state` to 1. Do not change any of the other parameters to `LassoCV` (i.e. leave them at their defaults).
5. Identify the best `alpha` value (the regularizer term, according to `sklearn`. In class, we refer to this as $\lambda$!) in terms of average mean squared error according to the cross-validation.
6. Finally, train a [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) model on the entire training dataset (`part1_train.csv`). You will use this to report the root mean squared error on the test set (Question 1.2.4 below), and use it in Part 1.3 below as well.

**Hint: The proceedure outlined above is very similar to ones we have discussed in class and shown how to do using `Pipeline`s.** 


In [3]:
# Write your code for Part 1.2 here
from sklearn.linear_model import LassoCV, Lasso
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Do the CV to find alpha
features = feature_sets[-1]
continuous = [feature for feature in features if feature in CONTINUOUS_FEATURES]
encoder = OneHotEncoder(sparse=False,drop='if_binary')
categorical = [feature for feature in features if feature in CATEGORICAL_FEATURES]
data_training = training_data[continuous+categorical]
print(data_training.shape)
data_test = test_data[continuous+categorical]
ct = make_column_transformer((scaler,continuous),(encoder,categorical),remainder='passthrough')
data_training_scaled = ct.fit_transform(data_training)
print(data_training_scaled.shape)
data_test_scaled = ct.transform(data_test) 
# l1_penalties = np.logspace(-3, 3, num=15)
regression_pipeline = LassoCV(random_state=1,cv=10) 
regression_pipeline.fit(data_training_scaled,y_training)

best_alpha = regression_pipeline.alpha_
train_rmse = np.sqrt(mean_squared_error(y_training,regression_pipeline.predict(data_training_scaled)))
test_rmse = np.sqrt(mean_squared_error(y_test, regression_pipeline.predict(data_test_scaled)))
print(test_rmse)
print(train_rmse)

(16270, 15)
(16270, 691)
1.1649797995719675
1.1229212879739903


Now, answer the following questions:
- **1.2.1** - How many total features are introduced by Step 2 above? Provide both the number and an explanation of how you got to this number.
- **1.2.2** - What was the best `alpha` value according to your cross-validation results?
- **1.2.3** - What was the **average RMSE** of the model with this `alpha` value on the k-fold cross validation on the *training* data?
- **1.2.4** - What was the **RMSE** of the model with this `alpha` value on the k-fold cross validation on the *test* data?


# Part 1.3 - Interpreting Model Coefficents

In this section we will interpret the coefficients from the final model you trained on all of the training data.

- **1.3.1** - How many non-zero coefficients are in this final model?
- **1.3.2** - What percentage of the coefficients are non-zero in this final model?
- **1.3.3** - Who were the three most critical review authors, as estimated by the model? How do you know?
- **1.3.4** - Who were the three artists that reviewers tended to like the most?  How do you know?
- **1.3.5** - What genre did Pitchfork reviewers tend to like the most? Which genre did they like the least?


In [4]:
# Code for 1.3 here
#1.3.1
coefficients = np.array(regression_pipeline.coef_)
print(np.count_nonzero(coefficients))
#1.3.2
print(np.count_nonzero(coefficients)/len(coefficients))
#1.3.3



484
0.7004341534008683


# Part 1.4 - "Manual" Cross-Validation + Holdout for Model Selection and Evaluation

We will finally use cross validation for both algorithm and model selection, with a hold-out test set for a final evaluation. We will use **5-fold cross validation** to identify the best parameters and hyperparameters for a set of models. We will then take our final models and use a final hold-out test set (the same one as above) to estimate the generalization error of the models.

Specifically, your task is first to write code that trains and evaluates the following models, one for each of the specified hyper parameters sets:

- `Decision Tree regression` - All combinations of a `max_depth` of 5, 10, or 20, and a `criterion` of `"squared error"` or `"absolute error"`. Use the [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor).
- Ridge regression - Use the following choices of L2 penalty: $[10^{-5}, 10^{-4}, ..., 10^4, 10^5]$. In Python, you can create a list of these numbers using `np.logspace(-5, 5, 11)`. Use the [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) class from sklearn to train a Ridge Regression model. The parameters you need to pass when constructing the Ridge model are `alpha`, which lets you specify what you want the L2 penalty to be, and `random_state=0` to avoid randomness.
- kNN regression - Values of `n_neighbors` of 1, 5, 10, and 15. Use the [KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) class.

Additional notes:
1. All models should use the feature sets described in Part 1.3 (the same ...one-hot encoded... categorical variables, and the scaled continuous variables)
2. As opposed to using the `KFold` class from `sklearn` like we did in class, we have instead provided you with pre-existing data sets; you should therefore use the pre-split data in the provided CSV files ``1.2_fold0.csv ... 1.2_fold4.csv``

**What to submit**:

1. Write out the result to a file called `part_1.4_results.csv` and submit this along with your assignment. The file should have the following columns:
- `model_name` - The name of the model, one of `DTR` (Decision Tree Regression), `Ridge`, or `KNN`.
- `hyperparameter_setting` - a column describing the hyperparameters of the model. For models with multiple hyperparameters, combine them using an underscore (you can do this with the code `"_".join(hyperparameters)`).
- `mean_training_rmse` - a column that gives the mean RMSE on the k-fold training data. You should take the average of the model’s errors on the different folds, using root mean squared error again as your evaluation metric.
- `sd_training_rmse` - a column that gives the standard deviation RMSE on the k-fold training data.
- `test_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the test data


2. Answer the following questions:
- **1.4.1** Report, for each model, the hyper parameter setting that resulted in the best performance
- **1.4.2** Which model performed the best overall on the cross-validation? 
- **1.4.3** Which model performed the best overall on the final test set? 
- **1.4.4** With respect to your answer for 1.4.3, why do you think that might be? (*Note: there is more than one correct way to answer this question*)
- **1.4.5** Which model/hyperparameter setting had the highest standard deviation across the different folds of the cross validation?
- **1.4.6** With respect to your answer for 1.4.6, why do you think that might be? (*Note: there is more than one correct way to answer this question*)

In [41]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
features=CONTINUOUS_FEATURES+CATEGORICAL_FEATURES
#preprocessor = ColumnTransformer(
#    transformers=[
#        ("num", numeric_transformer, selector(dtype_exclude="category")),
#        ("cat", categorical_transformer, selector(dtype_include="category")),#preprocessor = ColumnTransformer(
# def dt_pipe_function(depth):
#     return make_pipeline(new_pipeline, DecisionTreeRegressor(criterion="squared_error", max_depth=depth))

# def ridge_pipe_function(penalty):
#     return make_pipeline(new_pipeline, Ridge(alpha=penalty, random_state=0))

# def knn_pipe_function(neighbour):
#     make_pipeline(new_pipeline, KNeighborsRegressor(n_neighbors=neighbour))


neighbors = [1,5,10,15]
penalties = np.logspace(-5,5,11)
depths = [5,10,20]
new_pipeline=make_pipeline(ColumnTransformer([("num",scaler,CONTINUOUS_FEATURES),("cat",encoder,CATEGORICAL_FEATURES)]))
test0 = pd.read_csv("1.2_fold0.csv")
y_test_0 = test0['score']
test1 = pd.read_csv("1.2_fold1.csv")
y_test_1 = test1['score']
test2 = pd.read_csv("1.2_fold2.csv")
y_test_2 = test2['score']
test3 = pd.read_csv("1.2_fold3.csv")
y_test_3 = test3['score']
test4 = pd.read_csv("1.2_fold4.csv")
y_test_4 = test4['score']
tests = [test0,test1,test2,test3,test4]
y_tests = [y_test_0,y_test_1,y_test_2,y_test_3,y_test_4]

train_fold_0= pd.concat([test1,test2,test3,test4])
y_train_fold_0 = train_fold_0['score']
train_fold_1= pd.concat([test0,test2,test3,test4])
y_train_fold_1 = train_fold_1['score']
train_fold_2= pd.concat([test0,test1,test3,test4])
y_train_fold_2 = train_fold_2['score']
train_fold_3= pd.concat([test0,test1,test3,test4])
y_train_fold_3 = train_fold_3['score']
train_fold_4= pd.concat([test0,test1,test2,test3])
y_train_fold_4 = train_fold_4['score']
train_folds = [train_fold_0,train_fold_1,train_fold_2,train_fold_3,train_fold_4]
y_trains = [y_train_fold_0,y_train_fold_1,y_train_fold_2,y_train_fold_3,y_train_fold_4]
#decisiontree
# dt_pipeline = make_pipeline(new_pipeline, DecisionTreeRegressor(criterion="squared_error", max_depth=5))
# #ridge
# ridge_pipeline = make_pipeline(new_pipeline, Ridge(alpha=1, random_state=0))
# #knn
# KNN_pipeline = make_pipeline(new_pipeline, KNeighborsRegressor(n_neighbors=[]))
ridge_errors = {}
knn_value = []
dt_errors = {}
df_knn = pd.DataFrame(columns=['model_name','hyperparameter_setting','mean_training_rmse','sd_training_rmse','test_rmse'])
df_tree = pd.DataFrame(columns=['model_name','hyperparameter_setting','mean_training_rmse','sd_training_rmse','test_rmse'])
df_ridge = pd.DataFrame(columns=['model_name','hyperparameter_setting','mean_training_rmse','sd_training_rmse','test_rmse'])
for i in range(5):
    knn_errors = []
    
    for neighbor in neighbors:
        knn_pipeline = make_pipeline(new_pipeline, KNeighborsRegressor(n_neighbors=neighbor))
        knn_pipeline.fit(train_folds[i],y_trains[i])
        prediction_knn_train = knn_pipeline.predict(train_folds[i])
        error_knn_train = np.sqrt(mean_squared_error(y_trains[i],prediction_knn_train))
        prediction_knn_test = knn_pipeline.predict(tests[i])
        error_knn_test = np.sqrt(mean_squared_error(prediction_knn_test,y_tests[i]))
        sd_knn_train = np.std((prediction_knn_train-y_trains[i])**2)
        df_knn = df_knn.append({'model_name':'knn','hyperparameter_setting':neighbor,'mean_training_rmse':error_knn_train,'sd_training_rmse':sd_knn_train,'test_rmse':error_knn_test},ignore_index=True)
    
    for depth in depths:
        dt_pipeline = make_pipeline(new_pipeline,DecisionTreeRegressor(criterion="squared_error", max_depth=depth))
        dt_pipeline.fit(train_folds[i],y_trains[i])
        prediction_dt_train = dt_pipeline.predict(train_folds[i])
        error_dt_train = np.sqrt(mean_squared_error(y_trains[i],prediction_dt_train))
        prediction_dt_test = dt_pipeline.predict(tests[i])
        error_dt_test = np.sqrt(mean_squared_error(prediction_dt_test,y_tests[i]))
        sd_dt_train = np.std((prediction_dt_train-y_trains[i])**2)
        df_tree = df_tree.append({'model_name':'DTR','hyperparameter_setting':depth,'mean_training_rmse':error_dt_train,'sd_training_rmse':sd_dt_train,'test_rmse':error_dt_test},ignore_index=True)

    for penalty in penalties:
        ridge_pipeline = make_pipeline(new_pipeline, Ridge(alpha=penalty, random_state=0))
        ridge_pipeline.fit(train_folds[i],y_trains[i])
        prediction_ridge_train = ridge_pipeline.predict(train_folds[i])
        error_ridge_train = np.sqrt(mean_squared_error(y_trains[i],prediction_ridge_train))
        prediction_ridge_test = dt_pipeline.predict(tests[i])
        error_ridge_test = np.sqrt(mean_squared_error(prediction_ridge_test,y_tests[i]))
        sd_ridge_train = np.std((prediction_ridge_train-y_trains[i])**2)
        df_ridge = df_ridge.append({'model_name':'Ridge','hyperparameter_setting':penalty,'mean_training_rmse':error_ridge_train,'sd_training_rmse':sd_ridge_train,'test_rmse':error_ridge_test},ignore_index=True)
df_tree = df_tree.groupby(['model_name','hyperparameter_setting']).mean().reset_index()
df_knn = df_knn.groupby(['model_name','hyperparameter_setting']).mean().reset_index()
df_ridge = df_ridge.groupby(['model_name','hyperparameter_setting']).mean().reset_index()
df_total = pd.concat([df_tree, df_knn, df_ridge])
df_total.to_csv('part_1.4_results.csv')
# df_tree

C:\Users\17164\AppData\Local\Temp\ipykernel_26268\1155416211.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_knn = df_knn.append({'model_name':'knn','hyperparameter_setting':neighbor,'mean_training_rmse':error_knn_train,'sd_training_rmse':sd_knn_train,'test_rmse':error_knn_test},ignore_index=True)
C:\Users\17164\AppData\Local\Temp\ipykernel_26268\1155416211.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_knn = df_knn.append({'model_name':'knn','hyperparameter_setting':neighbor,'mean_training_rmse':error_knn_train,'sd_training_rmse':sd_knn_train,'test_rmse':error_knn_test},ignore_index=True)
C:\Users\17164\AppData\Local\Temp\ipykernel_26268\1155416211.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_kn

# Part 2

In class, we have shifted from regression to classification. Here, we're going to get a little practice in optimizing one of the classification models we saw in class - logistic regression. As a reminder...

The loss function of logistic regression (also known as the logistic-loss or log-loss) is given by:
\begin{equation}
  J({\bf w}) = \frac{1}{n}\sum_{i=1}^n \log{(1 + \exp{(-y_i{\bf w}^\top{\bf x}_i}))}
  \label{eqn:logloss}
\end{equation}

The gradient for this loss function, as derived in class, is:
\begin{equation}
  \nabla J({\bf w}) = -\frac{1}{n}\sum_{i=1}^n \frac{y_i}{1 + \exp{(y_i{\bf w}^\top{\bf x}_i)}}{\bf x}_i
  \label{eqn:loglossgradient}
\end{equation}


The Hessian for the loss function is given by:
\begin{equation}
  {\bf H}({\bf w}) = \frac{1}{n} \sum_{i=1}^n \frac{\exp{(y_i{\bf w}^\top{\bf x}_i)}}{(1 + \exp{(y_i{\bf w}^\top{\bf x}_i)})^2}{\bf x}_i{\bf x}_i^\top
  \label{eqn:loglosshessian}
\end{equation}

## Part 2.1 - Logistic Regression with Gradient Descent

In Part 2.1 we will implement logistic regression with gradient descent. You need to finish implementing 3 functions:

1. `logistic_objective` - compute the logistic loss for the given data set (see equation above)
2. `logistic_gradient` - compute the gradient vector of logistic loss for the given data set (see equation above)
3. `run_gradient_descent` - run the gradient descent algorithm, given these two functions.

We have provided you with some simulation data to evaluate your method with. Part 2.1 will, however, largely be graded by evaluating your code on a slightly different dataset to ensure robustness. 

In addition, please submit answers to the following questions on your written report:

- **2.1.1** - How did you go about selecting a good step size, i.e. one that was not too big or too small? (*Note: There is more than one correct answer to this*)
- **2.1.2** - What is the condition under which we assume that the gradient descent algorithm has converged in the code here?
- **2.1.3** - What is a different convergence metric we could have used? (*Note: There is more than one correct answer to this*)

In [168]:
import math
def logistic_objective(w, X, y):

    # compute log-loss error (scalar) with respect
    # to w (vector) for the given data X and y                               
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = scalar
    
    n = len(X)
#     error = np.log(1+np.exp(np.mutiply(-y,np.dot(w,np.transpose(X))))).mean()
#     print('The error from vectorization is ',error)
    error_for = 0
    for i in range(0,n):
        y_i = y[i]
        X_i = X[i]
        #print('The value of multiplicands are ',w.transpose().shape,X_i.shape)
        multi = np.dot(w.transpose(),X_i)
        print('The multi is ',multi,multi.shape)
        val = -1*y_i*np.dot(w,X_i)
        error_for+= math.log(1+np.exp(val))
#    print('The error for is ',error_for/n);
    return error_for/n


def logistic_gradient(w, X, y):

    # compute the gradient of the log-loss error (vector) with respect
    # to w (vector) for the given data X and y  
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = d length gradient vector (not a d x 1 matrix)
    res = np.array([0,0,0])
    for i in range(len(X)):
        y_i =  y[i]
        X_i = X[i]
        num = np.array(y_i*X_i)
        den = 1 + np.exp(y_i*np.dot(w,X_i))
        res = (res)+(num/den)

    return -1*(np.array(res)/len(X))

def run_gradient_descent(X,y):
    old_w = np.array([-1]*X.shape[1])
    # change this value! This is an unreasonable step size
    step_size = 0.001
    new_w = old_w - 1
    error = 1000000
    
    while ((new_w-old_w)**2).sum() > .0000000001:
        #IMPLEMENT THIS
        gradient = logistic_gradient(np.asarray(new_w),X,y)
        old_w = new_w
        new_w = np.subtract(new_w,(step_size * gradient))
        print('will the loop stop',((new_w-old_w)**2).sum())
        print('new w is ',new_w)
#       error = logistic_objective(np.asarray(new_w),X,y)
        #pass
#     print('The new weights is ',new_w)
    return new_w


In [169]:
from scipy.stats import uniform, bernoulli
import functools
draw_binary = functools.partial(np.random.binomial,n=1)

## Simulated data to test your method
DATA_SIZE = 10000
x1 = bernoulli(.5).rvs(DATA_SIZE)
x2 = np.floor(uniform(18,60).rvs(DATA_SIZE))
true_w = [-9, 3.5, 0.2]
xb = true_w[0] + true_w[1]*x1 + true_w[2]*x2
p = 1/(1 + np.exp(-xb))
y = np.array([1 if draw_binary(p=v) else -1 for v in p])

In [170]:
from sklearn.linear_model import LogisticRegression

# notice that logistic regression as implemented in sklearn does not get the exact results either!
# so you shouldn't worry if you're a bit off
X = np.hstack([np.ones((len(xb),1)), x1[:,np.newaxis], x2[:,np.newaxis]])
model = LogisticRegression(solver='liblinear', random_state=0,fit_intercept=False)
model.fit(X,y).coef_

array([[-8.68933043,  3.39691867,  0.19201559]])

In [171]:
# this is how we will test your results
gd_result = run_gradient_descent(X,y)
print(gd_result)
# is your result relatively close to the truth?
np.abs(true_w-gd_result).sum()


will the loop stop 0.0014001080312999947
new w is  [-1.9993215 -1.9995883 -1.9625904]
will the loop stop 0.0014001080312999947
new w is  [-1.998643  -1.9991766 -1.9251808]
will the loop stop 0.0014001080312999947
new w is  [-1.9979645 -1.9987649 -1.8877712]
will the loop stop 0.0014001080312999947
new w is  [-1.997286  -1.9983532 -1.8503616]
will the loop stop 0.0014001080312999947
new w is  [-1.9966075 -1.9979415 -1.812952 ]
will the loop stop 0.0014001080312999947
new w is  [-1.995929  -1.9975298 -1.7755424]
will the loop stop 0.0014001080312999947
new w is  [-1.9952505 -1.9971181 -1.7381328]
will the loop stop 0.0014001080312999947
new w is  [-1.994572  -1.9967064 -1.7007232]
will the loop stop 0.0014001080312999947
new w is  [-1.9938935 -1.9962947 -1.6633136]
will the loop stop 0.0014001080312999947
new w is  [-1.993215 -1.995883 -1.625904]
will the loop stop 0.0014001080312999947
new w is  [-1.9925365 -1.9954713 -1.5884944]
will the loop stop 0.0014001080312999947
new w is  [-1.99

KeyboardInterrupt: 

## <span style="color:red"> 574 Only</span> Part 2.2 - Optimization with Newton-Raphson <span style="color:red"> 574 Only</span>

In Part 2.2, you are going to, instead of using gradient descent, use the Newton-Raphson method to optimize the same logistic regression model. To do so, you will need to 1) implement the `logistic_hessian` function to compute the Hessian matrix of logistic loss for the given data set, and 2) use `scipy`'s `optimize` function to perform the optimization, rather than writing a function by hand to do so.  

For Part 2.2, you will only need to implement these functions, we will test them using our own code. You can, however, perform the same kinds of tests that we proposed above to check your work! 

In [13]:
def logistic_hessian(w, X, y):

    # compute the Hessian of the log-loss error (matrix) with respect
    # to w (vector) for the given data X and y                               
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # Hessian = d x d matrix
    
    
    if len(w.shape) == 1:
        w = w[:,np.newaxis]
        
        m = len(y)
        xw=np.dot(X,w)
        yxwt=np.matmul(xw.transpose(),y)
        predictions=np.exp(yxwt)
        xxt=np.matmul(X,X.transpose())
        predsqr=(1+predictions)*(1+predictions)
        hessian = (1/m)*((predictions*xxt)/predsqr)   
            
    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    #hessian = 0

    return hessian

In [18]:
from scipy.optimize import minimize

def run_newton_raphson(X,y):
    args = (X,y[:,np.newaxis])
    opts = {'maxiter' : 50}    # Preferred value.    
    w_init = np.zeros(X.shape[1])
    # note: this is almost what you need, you just need to figure out what arguments are necessary here!
    soln = minimize(#PUT_THE_RIGHT_ARGUMENTS_IN_HERE!
                    logistic_objective,
                    w_init,
                    hess=logistic_hessian(w_init, X, y),
                    x0=y,
                    args=args,
                    method='Newton-CG',
                    options=opts,
                    jac=none)
    w = w[:,np.newaxis]
    w = np.transpose(np.array(soln.x))
    return w

In [20]:
run_newton_raphson(X,y)

ValueError: ignored